In [1]:
import os
import json
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding


Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [12]:
DATA_DIR = './data'
MODEL_DIR = './model'

In [4]:
def save_weights(epoch, model):
    if not os.path.exists(MODEL_DIR):
        os.makedirs(MODEL_DIR)
    model.save_weights(os.path.join(MODEL_DIR, 'weights.{}.h5'.format(epoch)))



In [5]:
def load_weights(epoch, model):
    model.load_weights(os.path.join(MODEL_DIR, 'weights.{}.h5'.format(epoch)))

In [6]:
def build_sample_model(vocab_size):
    model = Sequential()
    model.add(Embedding(vocab_size, 512, batch_input_shape=(1, 1)))
    for i in range(3):
        model.add(LSTM(256, return_sequences=(i != 2), stateful=True))
        model.add(Dropout(0.2))

    model.add(Dense(vocab_size))
    model.add(Activation('softmax'))
    return model

In [9]:
def sample(epoch, header, num_chars):
    with open(os.path.join(DATA_DIR, 'char_to_idx.json')) as f:
        char_to_idx = json.load(f)
    idx_to_char = { i: ch for (ch, i) in char_to_idx.items() }
    vocab_size = len(char_to_idx)

    model = build_sample_model(vocab_size)
    load_weights(epoch, model)
    model.save(os.path.join(MODEL_DIR, 'model.{}.h5'.format(epoch)))

    sampled = [char_to_idx[c] for c in header]
    print(sampled)
    

    for i in range(num_chars):
        batch = np.zeros((1, 1))
        if sampled:
            batch[0, 0] = sampled[-1]
        else:
            batch[0, 0] = np.random.randint(vocab_size)
        result = model.predict_on_batch(batch).ravel()
        sample = np.random.choice(range(vocab_size), p=result)
        sampled.append(sample)

    return ''.join(idx_to_char[c] for c in sampled)

In [13]:
print(sample(100,"a",1024))

[58]
ag|"D"faf dfa|
"Em"gab "A7"aga|"Em"be'a "F#7"gfe|"Bm"fBd "F#7"edc|
"Bm"Bcd "B7/e"fed|"Em"edB "A7"AFD|"D"D3 "A7"=FDE|"D"DFA "C"=CEG|"D"D3 :|


X: 275
T:Scottish Reform
% Nottingham Music Database
Y:AABB
S:Kevin Briggs, via EF
M:6/8
K:G
P:A
|:d/2c/2|"G"B2B B^AB|"G"d2d d2B|"C"cBc Gce|"G"d2B G2:|
P:G
d|"G"dBG DGB|"G"DGB dcB|"C"cAG EGc|"G"BAG G2:|
P:B
g|"G"dBG GBd|"C"EFE "D7"A2G|"G/b"BAG "C"EGE|"D7"GED D2:|
P:B
|:d/2d/2|"G"dBg dBG|"C"cAG EGc|"C/e"GBc "D7"BcA|
"G"GBd gdB|"D"Add "A7"edc|"D"dcd "A7"edc|"D"d3 d2:|


X: 36
T:Cateaude Jig
% Nottingham Music Database
S:Trad, arr Phil Rowe
M:6/8
K:D
"A"A2A |:"D"DAc dee|"D"fgf "A"efe|"G"dcd "Em"Bcd|"A7"ecA Ace|
"A"age cde|"D"fgf fgf|"D"efd "A7"Ace|"D"d3 -d3:|


X: 144
T:Julian's Jig
% Nottingham Music Database
Y:AAB
S:Athole p 136, via EF
M:6/8
K:D
P:A
|:f/2e/2|"D"d2f fga|"D"f2a agf|"A7"ecA ecA|"D"A3 -A2||


X: 168
T:The Lark in the Clesfer
% Nottingham Music Database
S:acu Bowuch, via EF
M:6/8
K:G
dc|"G"B2B BAB|"C"c2e "D7"c2A|"G"Bcd "C"c2B|\
"